In [7]:
from __future__ import division
from scipy.stats import norm
from scipy.optimize import fsolve
from numpy import inf
from math import log, sqrt, exp
import pandas as pd
import datetime
from math import *


def black(cp=None, f=None, k=None, t=None, r=None, v=None, price=None, full=False, comp=inf):
    """ General Purpose BSM machine. Leave one parameter blank and that is the parameter that will
        be solved for.
            cp   = "c" for a call, anything else assumes a put
            f    = Forward Price of the underlying asset
            k    = Strike Price
            t    = Time until maturity (in years)
            r    = Interest rate
            v    = Implied volatility
            comp = How many times interest is compounded a year (default = inf, i.e. continous rates)
            full = If True, function returns a dictionary with price and all sensitivities
                   Otherwise only the calculated/calibrated parameter will be returned
    """

    D1 = lambda f, k, sigma, t: (log(f/k)+(sigma**2/2)*t)/(sigma*sqrt(t))
    D2 = lambda f, k, sigma, t: D1(f, k, sigma, t)-sigma*sqrt(t)

    if comp != inf:
        r = log((1+r/comp)**comp)   # convert rates
    optionType = 1 if cp.upper()[0] == "C" else -1   # If the first letter is not "c" or "C" it must be a put option
    parameters = locals().copy()

    def _black(cp, f, k, t, r, v, price, full, calibration=False, **kwargs):
        d1, d2 = D1(f, k, v, t), D2(f, k, v, t)
        price = optionType * exp(-r*t)*(f*norm.cdf(optionType*d1)-k*norm.cdf(optionType*d2))

        if not calibration and full:
            return {
                "price": price,
                "delta": optionType * norm.cdf(optionType*d1),
                "gamma": norm.pdf(d1)/(f*v*sqrt(t)),
                "vega": f*norm.pdf(d1)*sqrt(t)
            }
        else:
            return price

    def _calibrate(value, field, parameters):
        parameters.update({field: value[0]})
        return abs(_black(calibration=True, **parameters) - price, )

    missing = [a for a in ["f", "k", "t", "r", "v", "price"] if parameters[a] is None]

    if len(missing) > 1:
        raise Exception("Too many missing variables from: %s " % missing)
    if len(missing) == 0:
        raise Exception("All variables assigned - nothing to solve")

    if missing[0] != "price":
        # if we are missing any parameter different from price we need to calibrate
        result = fsolve(_calibrate, 0.1, args=(missing[0], parameters))

        if full is False:
            return result   # If full=False we simply return the calibrated parameter

    return _black(**parameters)



In [5]:
data = pd.read_pickle("data/BTC-29SEP23-30000-P.pkl")

In [17]:
years_to_maturity = (datetime.datetime(2023, 9, 29) - datetime.datetime.today()).days/365

In [6]:
data

,instrument_name,timestamp,mark_iv,mark_price,best_bid_price,best_ask_price,human_timestamp
0,BTC-29SEP23-30000-P,1666208971781,51.95,0.6183,0.0,0.0,2022-10-19 22:49:31.781000
16,BTC-29SEP23-30000-P,1666208972788,51.95,0.6183,0.0,0.0,2022-10-19 22:49:32.788000
31,BTC-29SEP23-30000-P,1666208973798,51.95,0.6183,0.0,0.0,2022-10-19 22:49:33.798000
46,BTC-29SEP23-30000-P,1666208974804,51.95,0.6183,0.0,0.0,2022-10-19 22:49:34.804000
61,BTC-29SEP23-30000-P,1666208975812,51.95,0.6183,0.0,0.0,2022-10-19 22:49:35.812000
...,...,...,...,...,...,...,...
9937,BTC-29SEP23-30000-P,1666209628350,48.15,0.6040,0.0,0.0,2022-10-19 23:00:28.350000
9952,BTC-29SEP23-30000-P,1666209629359,48.15,0.6040,0.0,0.0,2022-10-19 23:00:29.359000
9967,BTC-29SEP23-30000-P,1666209630363,47.80,0.6029,0.0,0.0,2022-10-19 23:00:30.363000
9982,BTC-29SEP23-30000-P,1666209631370,47.80,0.6029,0.0,0.0,2022-10-19 23:00:31.370000


In [20]:
ivs = []
for index, row in data.iterrows():
    iv = black("P", k=30000, f=row["mark_price"], t=years_to_maturity, r=0.04, price=100)
    ivs.append(iv)


In [21]:
ivs

[array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]),
 array([-1.15]